最初编写软件时的文件

可以在这个网站测试  
https://sketch.io/sketchpad/

In [1]:
from tkinter import Tk, Canvas, Button, PhotoImage
import keyboard
import pyautogui
from PIL import Image, ImageGrab
import win32api, win32gui
import time
import pickle
import numpy as np

跳过

In [ ]:
window = Tk()

window.geometry("650x537")
window.configure(bg = "#FFFFFF")

canvas = Canvas(
    window,
    bg = "#FFFFFF",
    height = 537,
    width = 650,
    bd = 0,
    highlightthickness = 0,
    relief = "ridge"
)
canvas.place(x = 0, y = 0)

def create_text(str):
    canvas.create_text(
        38.0,
        480.0,
        anchor="nw",
        text=str,
        fill="#080B1A",
        font=("Roboto", 24 * -1),
        tags=('print',)
    )

def button_clicked():
    canvas.delete('print')
    create_text("asd")

image_image = PhotoImage(
    file="./assets/bg.png")
image_1 = canvas.create_image(
    325.0,
    268.0,
    image=image_image
)

button_image = PhotoImage(
    file="./assets/button.png")
button = Button(
    image=button_image,
    borderwidth=0,
    highlightthickness=0,
    command=lambda: button_clicked(),
    relief="flat"
)
button.place(
    x=451.0,
    y=124.0,
    width=198.0,
    height=143.0
)
create_text("Print")


def start():
    print("Start")
def stop():
    print("Stop")

keyboard.add_hotkey('f11', start)
keyboard.add_hotkey('f12', stop)
    
window.resizable(False, False)
window.mainloop()

def get_image(name):
    hwnd = win32gui.FindWindow(0,name)
    if hwnd == 0:
        return None
    else:
        (x1,y1,x2,y2) = win32gui.GetWindowRect(hwnd)
        img = ImageGrab.grab((x1,y1,x2,y2))
        return img
img = get_image('Draw&Guess')
save_img(img)

Start
Start
Stop
Stop


In [56]:
a = []
def color():
    currentMouseX, currentMouseY = pyautogui.position()
    a.append(currentMouseX)
    a.append(currentMouseY)
    print(a)
keyboard.add_hotkey('f10', color)

<function keyboard.add_hotkey.<locals>.remove_()>

结束跳过

In [2]:
def save_img(v):
    filename = 'img.var'
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
def load_img():
    filename = 'img.var'
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r

In [3]:
img = load_img()
point_size = 4 # 6
# 绘画区域
width ,height = img.size
draw_area = 0.7
img1 = img.crop((width* (1-draw_area)/2, height* (1-draw_area)/2, width* (1+draw_area)/2, height* (1+draw_area)/2))
# 颜色板
color_board = img.crop((1470, 25, 1770, 125))
color_board = color_board.load()
# 调整调色板光标

In [4]:
diff_limit = 100
color_dic = {}
last_p = (255,255,255)
last_result = (1769, 124)

def colors_diff(pixel_a, pixel_b):
    return np.square(pixel_a[0]-pixel_b[0]) + np.square(pixel_a[1]-pixel_b[1]) + np.square(pixel_a[2]-pixel_b[2])

def find_color_all(p):
    diff_list = []
    index_list = []
    for x in range(300):
        for y in range(100):
            diff = colors_diff(p, color_board[x,y])
            if diff < 400:
                diff_list.append(diff)
                index_list.append((x,y))
    if len(diff_list) == 0:
        return (None, None)
    min_diff = min(diff_list)
    index = diff_list.index(min_diff)
    return index_list[index]
    
def find_color(p):
    global last_p
    global last_result
    # 白色,黄色
    if colors_diff((0,0,0),p) < diff_limit or colors_diff((160, 176, 192),p) < diff_limit:
        return (None, None)
    
    # 黑色
    if colors_diff((255,255,255),p) < diff_limit:
        return (1769, 124)
    
    # 上一个
    if colors_diff(p, last_p) < diff_limit:
        return last_result
    
    # 扫描字典
    if p in color_dic:
        return color_dic[p]
    
    # 完整扫描
    (x,y) = find_color_all(p)
    if x == None:
        return (None, None)
    else:
        result = (x + 1470,y +25)
        color_dic[p] = result
        last_p = p
        last_result = result
    return result

In [5]:
#读取图片 调整图片
target_img = Image.open('./example.jpg')
tiw, tih = target_img.size

#图片过大处理
if(tiw*draw_area>tih):
    if(tiw > width * draw_area):
        w = width * draw_area
        h = tih * width * draw_area / tiw
        target_img = target_img.resize((round(w),round(h)), Image.ANTIALIAS)
else:
    if(tih > height * draw_area):
        h = height * draw_area
        w = tiw * height * draw_area / tih
        target_img = target_img.resize((round(w),round(h)), Image.ANTIALIAS)
        
tiw, tih = target_img.size
start_x = round((width/2*draw_area)-(tiw/2)+(width* (1-draw_area)/2))
start_y = round((height/2*draw_area)-(tih/2)+(height* (1-draw_area)/2))

#图片缩小
tiw, tih = target_img.size
target_img = target_img.resize((round(tiw/point_size),round(tih/point_size)), Image.ANTIALIAS)
tiw, tih = target_img.size
target_img_p = target_img.load()

#检测是否聚焦

#选择最小画笔
# pyautogui.click(1407, 73)
# time.sleep(1)
# #移动到画板左上角
# pyautogui.moveTo(start_x+point_size/2, start_y+point_size/2)

# 循环画点
# pyautogui.click(200, 220)
# time.sleep(0.04)
# pyautogui.click(200, 220)

测试

In [16]:
ori_img = img.load()
from tqdm import tqdm
for x in tqdm(range(tiw)):
    for y in range(tih):
        (m,n) = find_color(target_img_p[x,y])
        if m!=None:
            target_img_p[x,y] = ori_img[m,n]

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:23<00:00,  8.55it/s]


In [35]:
color_dic = {}
for x in range(300):
        for y in range(100):
            color_dic[color_board[x,y]] = (x + 1470,y +25)
    
def find_color2(p):
    # 白色,黄色
    if colors_diff((0,0,0),p) < diff_limit or colors_diff((160, 176, 192),p) < diff_limit:
        return (None, None)
    
    # 黑色
    if colors_diff((255,255,255),p) < diff_limit:
        return (1769, 124)
    
    # 完整扫描
    (x,y) = find_color_all(p)
    if x == None:
        return (None, None)
    else:
        result = (x + 1470,y +25)
        color_dic[p] = result
        last_p = p
        last_result = result
    return result

In [12]:
colors = []
for x in range(300):
    for y in range(100):
        l = []
        l.append(color_board[x,y][0])
        l.append(color_board[x,y][1])
        l.append(color_board[x,y][2])
        colors.append(l)

In [17]:
colors = np.array(colors)
def closest(colors,color):
    color = np.array(color)
    distances = np.sqrt(np.sum((colors-color)**2,axis=1))
    index_of_smallest = np.where(distances==np.amin(distances))
    smallest_distance = colors[index_of_smallest]
    return smallest_distance

In [45]:
ori_img = img.load()
from tqdm import tqdm
for x in tqdm(range(tiw)):
    for y in range(tih):
        a = closest(colors,target_img_p[x,y])
        a = a[0]
        (m,n) = color_dic[(a[0],a[1],a[2])]
        target_img_p[x,y] = ori_img[m,n]

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:21<00:00,  9.14it/s]


In [23]:
ori_img = img.load()
from tqdm import tqdm
for x in tqdm(range(tiw)):
    for y in range(tih):
        (m,n) = find_color2(target_img_p[x,y])
        if m!=None:
            target_img_p[x,y] = ori_img[m,n]

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [08:15<00:00,  2.48s/it]
